In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, LongType, TimestampType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1667696313739_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

spark = create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
input_data = "s3a://udacity-dend/"
output_data = "s3://data-lake-project-hp/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
songSchema = StructType([
    StructField("artist_id",StringType()),
    StructField("artist_latitude",DoubleType()),
    StructField("artist_location",StringType()),
    StructField("artist_longitude",DoubleType()),
    StructField("artist_name",StringType()),
    StructField("duration",DoubleType()),
    StructField("num_songs",IntegerType()),
    StructField("song_id",StringType()),
    StructField("title",StringType()),
    StructField("year",IntegerType()),
])

# read song data file
song_data = input_data + 'song_data/*/*/*'
df = spark.read.json(song_data, schema=songSchema).drop_duplicates()
df.createOrReplaceTempView("songs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
songs_table = spark.sql("SELECT song_id, title, artist_id, year, duration FROM songs LIMIT 10")
songs_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOLLALT12A8C1399F3|Piano Concerto No...|ARWMEJW11F4C83C123|   0|319.37261|
|SOIEUCF12A6D4F9889|Symphony in B fla...|ARBB8QH1187FB54138|   0|398.34077|
|SODTEMK12AF72A7210|A Simple Desultor...|ARIWB161187B9AA1D5|1966|138.70975|
|SOMGIWL12A8C132D73|1776/He Plays the...|ARTAUG51187FB3E10D|   0| 185.0771|
|SOMHOLR12A6D4FBE6A|This Is Why I'm H...|ARGK9MB1187FB37402|   0|258.92526|
|SONSNAS12A58A796A9|Something To Be P...|ARLYWG51187FB52D19|2004| 255.4771|
|SOCMUUZ12A6310E299|I Wouldn't Normal...|ARA60941187B9AD4A3|1994|179.82649|
|SOPDHNT12A58A763A5|Of A Dear Friend ...|ARF1AUC1187B98D9A7|2003|297.09016|
|SOLKOEI12A6D4F85B9|Tomorrow's People...|AR86HDW1187FB37331|   0|422.26893|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
+-----------

In [33]:
songs_table = spark.sql("SELECT song_id, title, artist_id, year, duration FROM songs")
songs_table.write.parquet(path=output_data+"songs/songs_table.parquet", partitionBy=["year","artist_id"], mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
artists_table = spark.sql("SELECT artist_id, artist_name, artist_location, artist_latitude, artist_longitude FROM songs")
artists_table.write.parquet(path=output_data+"artists_table/artists_table.parquet", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
artists_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|AR7DQY51187B9B2D88|Chingo Bling w/ B...|               Texas|        31.1689|      -100.07715|
|AR5LQAT1187B9B1326|Nuova Compagnia D...|       Naples, Italy|        40.8399|        14.25185|
|ARDWI4G1187FB369E1|     Charlie Daniels|      Mt. Juliet, TN|       36.19524|       -86.51858|
|ARF1AUC1187B98D9A7|      Fighting Jacks|San Jose, California|       37.33847|      -121.88579|
|ARV3K201187FB3A680|       Helen Shapiro|Bethnal Green, Lo...|       51.52668|        -0.06257|
|ARRWVPI1187FB4615A|    The Hurt Process|   Brighton, England|       50.82821|        -0.13449|
|ARXGBPO1187B9ACA14|Andy Summers / Be...| Lancashire, England|       53.86121|        -2.56483|
|ARZ8S071187B993849|Tito Rodriguez_ L...

## Log Data

In [35]:
#log_data = input_data + 'log_data/*'
log_data = log_data = input_data + "log_data/*/*/*.json"

logSchema = StructType([
    StructField("artist",StringType()),
    StructField("auth",DoubleType()),
    StructField("firstName",StringType()),
    StructField("gender",StringType()),
    StructField("itemInSession",IntegerType()),
    StructField("lastName",StringType()),
    StructField("length",DoubleType()),
    StructField("level",StringType()),
    StructField("location",StringType()),
    StructField("method",StringType()),
    StructField("page",StringType()),
    StructField("registration",DoubleType()),
    StructField("sessionId",IntegerType()),
    StructField("song",StringType()),
    StructField("status",IntegerType()),
    StructField("ts",LongType()),
    StructField("userAgent",StringType()),
    StructField("userId",IntegerType()),
])

df = spark.read.json(log_data).drop_duplicates() #, schema=logSchema
#df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df.createOrReplaceTempView("logs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
logs_table = spark.sql("SELECT * FROM logs WHERE page='NextSong'")
logs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------+----------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|           artist|     auth| firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+-----------------+---------+----------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|       Bananarama|Logged In|Jacqueline|     F|           19|   Lynch| 219.0624| paid|Atlanta-Sandy Spr...|   PUT|NextSong|1.540223723796E12|      556|               Venus|   200|1542141364796|"Mozilla/5.0 (Mac...|    29|
|Animal Collective|Logged In|    Aleena|     F|           11|   Kirby|269.81832| paid|Waterloo-Cedar Fa...|   PU

In [38]:
users_table = spark.sql("SELECT userId, firstName, lastName, gender, level FROM logs WHERE page='NextSong'")
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    49|    Chloe|  Cuevas|     F| paid|
|    49|    Chloe|  Cuevas|     F| paid|
|    85|  Kinsley|   Young|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    42|   Harper| Barrett|     M| paid|
+------+---------+--------+------+-----+
only showing top 5 rows

In [39]:
# write users table to parquet files
users_table.write.parquet(path=output_data+"users/users_table.parquet", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Time table

In [40]:
import datetime
# create timestamp column from original timestamp column
to_timestamp_udf = udf(lambda ts_string: datetime.datetime.fromtimestamp(ts_string / 1000.0),TimestampType())
spark.udf.register("to_timestamp", to_timestamp_udf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7f8b5bf1bd40>

In [41]:
time_table = spark.sql(
'''
SELECT 
  ts, 
  hour(to_timestamp(ts)) AS hour,
  day(to_timestamp(ts)) AS day,
  weekofyear(to_timestamp(ts)) AS week,
  month(to_timestamp(ts)) AS month,
  year(to_timestamp(ts)) AS year,
  weekday(to_timestamp(ts)) AS weekday
FROM logs
'''
          )
time_table = time_table.withColumn('start_time', to_timestamp_udf('ts').cast(TimestampType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
time_table.write.parquet(path=output_data+"time/time_table.parquet", mode="overwrite", partitionBy=["year","month"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----+---+----+-----+----+-------+--------------------+
|           ts|hour|day|week|month|year|weekday|          start_time|
+-------------+----+---+----+-----+----+-------+--------------------+
|1542141364796|  20| 13|  46|   11|2018|      1|2018-11-13 20:36:...|
|1541383518796|   2|  5|  45|   11|2018|      0|2018-11-05 02:05:...|
|1543398383796|   9| 28|  48|   11|2018|      2|2018-11-28 09:46:...|
|1542119097796|  14| 13|  46|   11|2018|      1|2018-11-13 14:24:...|
|1541415367796|  10|  5|  45|   11|2018|      0|2018-11-05 10:56:...|
+-------------+----+---+----+-----+----+-------+--------------------+
only showing top 5 rows

## TO-DO

In [44]:
# read in song data to use for songplays table
song_data = input_data +'song_data/*/*/*'
song_df = spark.read.json(song_data, schema=songSchema).drop_duplicates()
song_df.createOrReplaceTempView("songs")
songs_table = spark.sql("SELECT song_id, title, artist_id, year, duration FROM songs")
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOLLALT12A8C1399F3|Piano Concerto No...|ARWMEJW11F4C83C123|   0|319.37261|
|SOIEUCF12A6D4F9889|Symphony in B fla...|ARBB8QH1187FB54138|   0|398.34077|
|SODTEMK12AF72A7210|A Simple Desultor...|ARIWB161187B9AA1D5|1966|138.70975|
|SOMGIWL12A8C132D73|1776/He Plays the...|ARTAUG51187FB3E10D|   0| 185.0771|
|SOMHOLR12A6D4FBE6A|This Is Why I'm H...|ARGK9MB1187FB37402|   0|258.92526|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

In [51]:
# extract columns from joined song and log datasets to create songplays table
from pyspark.sql import functions as F

songplays_table = spark.sql('''
SELECT 
  to_timestamp(l.ts) AS start_time, 
  l.userId, 
  l.level, 
  s.song_id, 
  s.artist_id, 
  l.sessionId, 
  l.location, 
  l.userAgent,
  year(to_timestamp(l.ts)) AS year,
  month(to_timestamp(l.ts)) AS month
FROM 
  logs l 
  INNER JOIN songs s ON l.song = s.title AND l.artist = s.artist_name 
ORDER BY 
  l.ts 
''')
songplays_table = songplays_table.withColumn('songplay_id', F.monotonically_increasing_id())
#songplays_table = songplays_table.withColumn('start_time', to_timestamp_udf('ts').cast(TimestampType()))
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+-----------+
|          start_time|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|year|month|songplay_id|
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+-----------+
|2018-11-01 21:11:...|     8| free|SOEIQUY12AF72A086A|ARHUC691187B9AD27F|      139|Phoenix-Mesa-Scot...|"Mozilla/5.0 (Win...|2018|   11|          0|
|2018-11-02 16:35:...|    50| free|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|      156|New Haven-Milford...|"Mozilla/5.0 (Win...|2018|   11|          1|
|2018-11-02 17:31:...|    10| free|SOHTKMO12AB01843B0|AR5EYTL1187B98EDA0|      182|Washington-Arling...|"Mozilla/5.0 (Mac...|2018|   11|          2|
|2018-11-02 18:02:...|    50| free|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|      207|New Haven-Milford...|"M

In [52]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.parquet(path=output_data+"songplays/songplays.parquet", mode="overwrite", partitionBy=["year","month"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…